**Overview**

This dataset consists of aviation accident survey data, which includes information on whether the aircraft was destroyed, the type of injuries sustained by individuals, the location of the accidents along with the year and month, the type of engine involved, weather conditions, the purpose of the flight, and other relevant details.

**Business Problem**

This analysis aims to identify patterns and trends in aviation accidents to enhance safety measures. By understanding the relationship between weather conditions, engine types, injury severity, and flight purpose, we can develop insights to prevent future accidents. The findings will support data-driven decision-making in aviation safety policies and operational practices. Ultimately, this research seeks to improve overall flight safety and reduce accident-related risks

**Data Understanding**

In this data analysis, the first step involves importing the necessary libraries and loading the dataset to begin the analysis. Next, I will handle any missing values to ensure the data is complete and reliable. Following this, I will transform the data to enhance its interpretability, such as grouping related categories into broader groups for better clarity. The data will then be explored through various visualizations to uncover relationships and patterns between key variables, such as weather conditions, injury severity, and flight purposes. Finally, I will summarize the findings and provide conclusions that offer insights into aviation safety and accident prevention.

**DATA AND LIBRARY IMPORTATION**

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

In [ ]:
# Loading dataset
df = pd.read_csv('AviationData.csv', encoding='latin-1')
df.head()

**DATA** **UNDERSTANDING**



In [ ]:
# understanding basic information of the dataset
df.info()

In [ ]:
# creating a copy of my original data so as not to bring any modification
df1 = df.copy()


**HANDLING** **MISSING** **VALUES**


In [ ]:
# dropping columns that have a lot of missing values
df1.drop(columns=['Latitude'], inplace=True)
df1.drop(columns=['Longitude'],inplace=True)
df1.drop(columns=['Airport.Code'],inplace=True)
df1.drop(columns=['Airport.Name'],inplace=True)
df1.drop(columns=['Aircraft.Category'],inplace=True)
df1.drop(columns=['FAR.Description'],inplace=True)
df1.drop(columns=['Schedule'],inplace=True)
df1.drop(columns=['Air.carrier'],inplace=True)

In [ ]:
df1.info()

In [ ]:
# checking the number of missing value
df1.isnull().sum()

**DATA** **CLEANING**

In [ ]:
# further columns removal
columns_name = ['Publication.Date','Broad.phase.of.flight','Engine.Type']
df1.drop(columns=columns_name,inplace=True)

In [ ]:
df1.info()

In [ ]:
# creating a subset with only numerical columns
df1_num = df1.select_dtypes(include=['number'])

In [ ]:
df1_num.corr()

In [ ]:
# ploting the correlation between the different columns
sns.pairplot(df1_num)

From this plots i can understand that the correlation betweeen the columns is poor,so the correlation can not be used to handle missing values


In [ ]:
# in order to handle the missing values, the mean,median and mode for each column is computed so as to understand the distribution of the data

mean1=df1_num['Total.Fatal.Injuries'].mean()
median1=df1_num['Total.Fatal.Injuries'].median()
mode1=df1_num['Total.Fatal.Injuries'].mode()[0]
print(f"Mean: {mean1}, Median: {median1}, Mode: {mode1}")

In [ ]:
mean2=df1_num['Total.Serious.Injuries'].mean()
median2=df1_num['Total.Serious.Injuries'].median()
mode2=df1_num['Total.Serious.Injuries'].mode()[0]
print(f"Mean: {mean2}, Median: {median2}, Mode: {mode2}")

In [ ]:
mean3=df1_num['Total.Minor.Injuries'].mean()
median3=df1_num['Total.Minor.Injuries'].median()
mode3=df1_num['Total.Minor.Injuries'].mode()[0]
print(f"Mean: {mean3}, Median: {median3}, Mode: {mode3}")

In [ ]:
mean4=df1_num['Total.Uninjured'].mean()
median4=df1_num['Total.Uninjured'].median()
mode4=df1_num['Total.Uninjured'].mode()[0]
print(f"Mean: {mean4}, Median: {median4}, Mode: {mode4}")

In [ ]:
df1_num['Total.Fatal.Injuries'].fillna(median1, inplace=True)
df1_num['Total.Serious.Injuries'].fillna(median2, inplace=True)
df1_num['Total.Minor.Injuries'].fillna(median3, inplace=True)
df1_num['Total.Uninjured'].fillna(median4, inplace=True)

In [ ]:
df1_num.isnull().sum()

In [ ]:
# I am returning back to the data frame the numerical columns,where the missing value has just been handled

columns_to_replace = ['Total.Fatal.Injuries', 'Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured']
df1[columns_to_replace] = df1_num[columns_to_replace]


In [ ]:
df1.isnull().sum()

In [ ]:
df1.info()

In [ ]:
# Am checking missing value in the columns if they share the same row

num_nulls = df1.isnull().sum(axis=1)
rows_with_nulls = df1[num_nulls == 9].index
rows_with_nulls
df1= df1.drop(index=rows_with_nulls )
df1.isnull().sum()


**DATA** **HANDLING**

In [ ]:
# i want to convert the Event.Date to seperate columns year,month,day. So am doing this to compare it year wise,month wise and day wise

df1['Event.Date'] = pd.to_datetime(df['Event.Date'])
df1['Event.Year'] = df1['Event.Date'].dt.year
df1['Event.Month'] = df1['Event.Date'].dt.month
df1['Event.Day'] = df1['Event.Date'].dt.day

In [ ]:
# I want to change the Injury.Severity column. When the value is fetal,the number of people injured is written with it so am going to seperate them.

df1['num_injured'] = df1['Injury.Severity'].str.extract(r'\((\d+)\)').astype(float)
df1['Injury.Severity'] = df1['Injury.Severity'].str.replace(r'\(\d+\)', '', regex=True).str.strip()
df1['Injury.Severity'].unique()

In [ ]:
# So i decided to change the 'Serious' injuries to Fatal while the 'Minor' and 'Incident' to NOn-Fetal. Incident was considered as minor because the corrisponding value in Aircraft.damage is minor.
# The purpuse of this is to reduce the categories.

df1['Injury.Severity'] = df1['Injury.Severity'].replace({
    'Serious': 'Fatal',
    'Incident': 'Non-Fatal',
    'Minor': 'Non-Fatal',
    'Unavailable': np.nan
})
df1['Injury.Severity'].unique()

In [ ]:
df1['Aircraft.damage'].unique()

In [ ]:
# I am going to consider the Substaintial and Minor as Non destroyed,Unknown is going to be considered as nan.

df1["Aircraft.damage"] = df1["Aircraft.damage"].replace({
    'Minor': 'Non-Destroyed',
    'Substantial': 'Non-Destroyed',
    'Unknown': np.nan
})
df1['Aircraft.damage'].unique()

In [ ]:
df1['Purpose.of.flight'].unique()

In [ ]:
# Since Purpose of has many categorical values, i am going to arrenge them in to 7 categories
category_mapping = {
    'Personal': 'Personal/Business',
    'Business': 'Personal/Business',
    'Executive/corporate': 'Personal/Business',
    'Other Work Use': 'Personal/Business',
    'Positioning': 'Ferry/Positioning',
    'Instructional': 'Flight Training/Testing',
    'Flight Test': 'Flight Training/Testing',
    'Unknown': np.nan,
    'Ferry': 'Ferry/Positioning',
    'Aerial Observation': 'Aerial Work',
    'Aerial Application': 'Aerial Work',
    'Public Aircraft': 'Public Aircraft',
    'Skydiving': 'Recreational/Sport',
    'Air Race/show': 'Recreational/Sport',
    'Air Race show': 'Recreational/Sport',
    'Air Drop': 'Aerial Work',
    'Public Aircraft - Federal': 'Public Aircraft',
    'Glider Tow': 'Aerial Work',
    'Public Aircraft - Local': 'Public Aircraft',
    'External Load': 'Aerial Work',
    'Public Aircraft - State': 'Public Aircraft',
    'Banner Tow': 'Aerial Work',
    'Firefighting': 'Aerial Work',
    'ASHO': 'Recreational/Sport',
    'PUBS': 'Public Aircraft',
    'PUBL': 'Public Aircraft'
}
df1['Purpose.of.flight'] = df1['Purpose.of.flight'].replace(category_mapping)
df1['Purpose.of.flight'].unique()

In [ ]:
df1['Weather.Condition'].unique()

In [ ]:
# Am changing the UNK value in nan in Weather Condition column

df1['Weather.Condition'] = df1['Weather.Condition'].replace({
    'Unk': np.nan,
    'UNK': np.nan ,
    'Unavailable': np.nan
})
df1['Weather.Condition'].unique()

In [ ]:
# The Report.status column is removed because it has entry that has long sentances.

df1.drop(['Report.Status'],axis=1 ,inplace=True)

In [ ]:
df1.info()

In [ ]:

# i am filling all the nan values with UNK
df1.fillna("UNK", inplace=True)

In [ ]:
df1.drop(['num_injured'],axis=1, inplace=True)

In [ ]:
# The value for location is seperated into city and state

df1['City'] = df['Location'].str.split(',').str[0]
df1['State'] = df['Location'].str.split(',').str[1]

**DATA VISUALIZATION**

In [ ]:
#In this code i am trying to understand what is the weather like during accidents
weather_condition_counts = df1['Weather.Condition'].value_counts()
sns.barplot(
    x=weather_condition_counts.values,
    y=weather_condition_counts.index,
    palette="coolwarm",
    hue=weather_condition_counts.index,
    dodge=False,
    legend=False
)
plt.xlabel('Count')
plt.ylabel('Weather Condition')
plt.title('Count of Weather Condition Type During Accident')
plt.show()


conclusion:i observed that most of the accident occur when the weather condition is VMC meaning the visibility is good.

In [ ]:
# in this plot i will count the type of weather condition for each type of injury so as to understand what kind of weather prevails for different degree of injury

injury_columns = ['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, column in enumerate(injury_columns):
    injury_counts_by_weather = df1.groupby('Weather.Condition')[column].count()
    sns.barplot(
        x=injury_counts_by_weather.index,
        y=injury_counts_by_weather.values,
        ax=axes[i // 2, i % 2],
        palette="coolwarm",
        hue=injury_counts_by_weather.index,
        legend=False
    )
    axes[i // 2, i % 2].set_title(f'Weather Condition Count for {column}')
    axes[i // 2, i % 2].set_xlabel('Weather Condition')
    axes[i // 2, i % 2].set_ylabel('Count')
    axes[i // 2, i % 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

**conclusion**: it can be seen that for all the injury the weather condition is VMC

In [ ]:
df1['Weather.Condition'].unique()

In [ ]:
#i want to see the relationship of engine type with the injury type, so as to understand which engine type result in major

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, column in enumerate(injury_columns):
    injury_counts_by_engines = df1.groupby('Number.of.Engines')[column].count()
    sns.barplot(
        x=injury_counts_by_engines.values,
        y=injury_counts_by_engines.index,
        ax=axes[i // 2, i % 2],
        palette="coolwarm",
        hue=injury_counts_by_engines.index,
        legend=False
    )
    axes[i // 2, i % 2].set_title(f'Injury Count for {column} by Number of Engines')
    axes[i // 2, i % 2].set_xlabel('Count')
    axes[i // 2, i % 2].set_ylabel('Number of Engines')
    axes[i // 2, i % 2].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()


**conclusion**:engine 1 has the highest number of accidents for the different injuries.

In [ ]:
#i want to see the relation between type ofinjury and the purpose of flight, to see in what kind of purposes the accident is fatal and in which is just minor

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, column in enumerate(injury_columns):
    injury_counts_by_purpose = df1.groupby('Purpose.of.flight')[column].count()
    sns.barplot(
        x=injury_counts_by_purpose.values,
        y=injury_counts_by_purpose.index,
        ax=axes[i // 2, i % 2],
        palette="coolwarm",
        hue=injury_counts_by_purpose.index,
        legend=False
    )
    axes[i // 2, i % 2].set_title(f'Injury Count for {column} by Purpose of Flight')
    axes[i // 2, i % 2].set_xlabel('Count')
    axes[i // 2, i % 2].set_ylabel('Purpose of Flight')
    axes[i // 2, i % 2].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()

**conclusion**: most of the injury occur for personal or business. substatial injuries also occur for flight training or testing purpose.

In [ ]:
#I want to see the relation between the injury type and the degree of damage of the aircraft, so as to see if there is a relation between fatal injury and destroyed aircraft.

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, column in enumerate(injury_columns):
    injury_counts_by_damage = df1.groupby('Aircraft.damage')[column].count()
    sns.barplot(
        x=injury_counts_by_damage.values,
        y=injury_counts_by_damage.index,
        ax=axes[i // 2, i % 2],
        palette="coolwarm",
        hue=injury_counts_by_damage.index,
        dodge=False,
        legend=False
    )
    axes[i // 2, i % 2].set_title(f'Injury Count for {column} by Aircraft Damage')
    axes[i // 2, i % 2].set_xlabel('Count')
    axes[i // 2, i % 2].set_ylabel('Aircraft Damage')
    axes[i // 2, i % 2].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()

**conclusion**: most of the aircraft were not destroyed in relation to the different injury that occured.

In [ ]:
# i want to see the relationship between top 10 country and type of injury, so as to understand which country has highest number of different type of injury

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, column in enumerate(injury_columns):
    injury_counts_by_country = df1.groupby('Country')[column].count()
    top_countries = injury_counts_by_country.nlargest(10)
    sns.barplot(
        x=top_countries.values,
        y=top_countries.index,
        ax=axes[i // 2, i % 2],
        palette="coolwarm",
        hue=top_countries.index,
        dodge=False,
        legend=False
    )
    axes[i // 2, i % 2].set_title(f'Injury Count for {column} by Country')
    axes[i // 2, i % 2].set_xlabel('Count')
    axes[i // 2, i % 2].set_ylabel('Country')
    axes[i // 2, i % 2].tick_params(axis='y', rotation=45)

plt.tight_layout()
plt.show()

**conclusion**: it can be seen that USA has the highest count  in number of accidents for different injury types.

In [ ]:
# i want to see the number of accidents for different states in the USA since the country has the highest accidents
usa_accidents = df1[df1['Country'] == 'United States']
accidents_by_state = usa_accidents.groupby('Location').size().nlargest(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=accidents_by_state.values, y=accidents_by_state.index, palette="coolwarm", hue=accidents_by_state.index)

plt.xlabel('Number of Accidents')
plt.ylabel('State')
plt.title('Top 10 States with the Highest Number of Accidents (United States)')
plt.tight_layout()
plt.show()

**conclusion**: Anchorage state has the highest accidents

In [ ]:
# i want to see the number of accident per year so as to know which years have high value
accidents_per_year = df1.groupby('Event.Year').size()

plt.figure(figsize=(10, 6))
sns.lineplot(x=accidents_per_year.index, y=accidents_per_year.values, marker='o', color='b')

plt.xlabel('Year')
plt.ylabel('Number of Accidents')
plt.title('Number of Accidents per Year')

plt.tight_layout()
plt.show()

**conclusion**: there are a lot of accidents after 1980

In [ ]:
#To visualize the number of accidents by month from the year 1980 onward, to know which month have high accidents
df_filtered = df1[df1['Event.Year'] >= 1980]
accidents_by_month = df_filtered.groupby(['Event.Year', 'Event.Month']).size().unstack(fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(accidents_by_month, cmap='YlGnBu', annot=True, fmt='d', cbar_kws={'label': 'Number of Accidents'}, linewidths=0.5)

plt.xlabel('Month')
plt.ylabel('Year')
plt.title('Heatmap of Number of Accidents by Month (From 1980 Onward)')
plt.xticks(ticks=range(12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.tight_layout()
plt.show()

**conclusion**: summer months show more accidents form 1980 onward. It is especially high for late years than recent ones.

In [ ]:
# i want to see the number of accidents per company maker so as to know which company is not doing a good job
accidents_by_make = df1.groupby('Make').size().nlargest(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=accidents_by_make.values, y=accidents_by_make.index, palette="coolwarm", hue=accidents_by_make.index)

plt.xlabel('Number of Accidents')
plt.ylabel('Company Maker')
plt.title('Top 10 Company Makers with the Highest Number of Accidents')
plt.tight_layout()
plt.show()

**conclusion**: Cessna has the highest record of accidents followed by Piper.

In [ ]:
#i want to see the relationship between the number of accidents for Cessna and Piper, and how this varies over the years from 1980 onwards
filtered_data = df1[(df1['Make'].isin(['Cessna', 'Piper'])) & (df1['Event.Year'] >= 1980)]

accidents_by_year = filtered_data.groupby(['Event.Year', 'Make']).size().unstack().fillna(0)

plt.figure(figsize=(12, 6))
accidents_by_year.plot(kind='line', marker='o', color=sns.color_palette("coolwarm", 2))

plt.xlabel('Year')
plt.ylabel('Number of Accidents')
plt.title('Number of Accidents for Cessna and Piper Over the Years (1980 and Beyond)')
plt.tight_layout()
plt.show()

**conclusion**: it can be seen that the accidents were slowly decreasing from 1980 onward, but around 2016 a suddent increase occured which slowly decreased till  2022

**Conclusion**

Weather Condition: VMC (Visual Meteorological Conditions) is the most common weather condition for accidents across all injury types.

Engine Type: Single-engine aircraft (1 engine) are involved in the majority of accidents.

Purpose of Flight: Most accidents occur during personal or business flights.

Aircraft Damage: The majority of accidents do not result in severe aircraft destruction.

Geographic Location: The USA has the highest number of recorded accidents, with Anchorage having the highest accident rate among states.

Trends Over Time: A significant number of accidents have occurred since 1980, with a descending trend in recent years.

Seasonal Pattern: The highest number of accidents occur during the summer months.

Aircraft Make: Cessna is the maker with the highest number of accidents, followed by Piper.





